In [ ]:
!pip install gdown

In [ ]:
import os
import zipfile
import gdown

# Téléchargement du dataset
Téléchargement de 800 images

In [ ]:
# URL de téléchargement direct Google Drive
# pour le gros dataset non tagged
url = 'https://drive.google.com/uc?id=16rYRrxUXpGyPWVq5uhgYzNlZd6hsGX7-'
# pour le petit dataset pokemon
# url = 'https://drive.google.com/uc?id=1yfy6XXv0VikxR8xTuz-xQts_MmGWRDUy'
# Chemin de destination
output = 'dataset.zip'

# Télécharge le fichier depuis l'URL
gdown.download(url, output, quiet=False)

# Décompresse le fichier dans le dossier 'img'
with zipfile.ZipFile(output, 'r') as zip_ref:
    zip_ref.extractall('images')
# Supprime le fichier zip si désiré
os.remove(output)

# Création du fichier de métadonnées

In [ ]:
!pip install pillow scikit-learn

In [ ]:
import json
import os

import numpy as np
from PIL import Image, ExifTags
from sklearn.cluster import MiniBatchKMeans

In [ ]:
def find_dominant_color(image_path, n_clusters=2):
    img = Image.open(image_path).convert('RGBA')  # Convertir en RGBA
    img = img.resize((50, 50))  # Optionnel: Redimensionner

    # Convertir l'image RGBA en une array numpy, ignorer les pixels complètement transparents
    numarray = np.array(img)
    numarray = numarray[:, :, :3][numarray[:, :, 3] > 0]  # Ignorer les pixels transparents

    clusters = MiniBatchKMeans(n_clusters=n_clusters)
    clusters.fit(numarray)

    counts = np.bincount(clusters.labels_)
    most_frequent = clusters.cluster_centers_[counts.argmax()]

    return tuple(int(c) for c in most_frequent)


# Fonction pour avoir l’orientation de l’image
def get_image_orientation(img):
    if img.width > img.height:
        return "paysage"
    elif img.width < img.height:
        return "portrait"
    else:
        return "carre"


# Fonction pour extraire les données Exif
def get_exif_data(img):
    exif_data = {}
    raw_exif = img._getexif()
    if raw_exif:
        for tag, value in raw_exif.items():
            decoded_tag = ExifTags.TAGS.get(tag, tag)
            exif_data[decoded_tag] = value
    return exif_data

# Chemin vers le dossier d'images et le fichier de métadonnées
images_path = 'images/unsplash-images-collection'
metadata_file = 'image_metadata.json'
metadata = []

for image_file in os.listdir(images_path):
    image_path = os.path.join(images_path, image_file)

    img = Image.open(image_path)
    # Extraction des données Exif
    exif_data = get_exif_data(img)
    dominant_color = find_dominant_color(image_path)  # Trouver la couleur dominante

    metadata.append({
        "nom": image_file,
        "taille": img.size,
        "format": img.format,
        "orientation": get_image_orientation(img),
        "exif": exif_data,  # Ajoutez les données Exif ici
        "couleur_dominante": dominant_color,  # Ajouter la couleur dominante aux métadonnées

        "tags": [],
        "favori": "n/a"
    })

if metadata:
    with open(metadata_file, 'w') as f:
        json.dump(metadata, f, indent=4)
    print("Métadonnées enregistrées.")
else:
    print("Aucune métadonnée à enregistrer.")

# Création automatisée de tags

## Version CPU
Compatible avec toutes les configurations

In [ ]:
!pip install tensorflow sentence-transformers

## Version GPU
Pour les utilisateurs Linux disposant d'un GPU NVIDIA uniquement.
Il est nécessaire d'avoir installé les drivers CUDA et cuDNN au préalable.

Pour les utilisateurs de Windows, il est possible d'installer TensorFlow en mode GPU avec Anaconda.
Installer cudatoolkit=11.2 cudnn=8.1.0
Une fois installé, installer tensorflow=2.10 avec pip et sentence-transformers.

In [ ]:
if os.name == 'posix':
    !pip install tensorflow[and-cuda]
    !pip install sentence-transformers

In [ ]:
import json

from tensorflow.keras.applications import InceptionV3
from tensorflow.keras.applications.inception_v3 import preprocess_input, decode_predictions
from tensorflow.keras.preprocessing import image
from sentence_transformers import SentenceTransformer, util

In [ ]:
def predict_tags(img_path, model):
    img = image.load_img(img_path, target_size=(299, 299))
    x = image.img_to_array(img)
    x = np.expand_dims(x, axis=0)
    x = preprocess_input(x)
    preds = model.predict(x)
    # Retourne une liste des top 3 tags prédits
    return [tag[1] for tag in decode_predictions(preds, top=1)[0]]

def associated_theme(tag,model_language,theme_embeddings):
        # Charger le modèle de transformation de phrases

    tag_embedding = model_language.encode([tag], convert_to_tensor=True)
    max_similarity = -1
    assigned_theme = "other"
    for theme, theme_embedding in theme_embeddings.items():
        similarity = util.pytorch_cos_sim(tag_embedding, theme_embedding).max()
        if similarity > max_similarity:
            max_similarity = similarity
            assigned_theme = theme
    return assigned_theme

# Charger les métadonnées existantes
def load_metadata():
    metadata_file = 'image_metadata.json'
    with open(metadata_file, 'r') as f:
        return json.load(f)


# Chemin vers le dossier d'images et le fichier de métadonnées
image_folder = 'images/unsplash-images-collection'

"""Mise à jour des métadonnées avec des tags prédits pour chaque image."""
metadata = load_metadata()
metadata_file = 'image_metadata.json'
image_files = [file for file in os.listdir(image_folder) if
               file.lower().endswith(('.png', '.jpg', '.jpeg', '.gif'))]
model = InceptionV3(weights='imagenet')

model_language = SentenceTransformer('all-MiniLM-L6-v2')

# Thèmes et mots-clés correspondants
themes = {
    "nature": ["forest","greenhouse","boathouse", "river", "mountain", "sky", "lake", "ocean", "canoe", "geyser", "flower", "maze","alp","dam","snowplow", "desert",
               "valley","hay","bubble"],
    "sports": ["mountain_bike","soccer", "basketball", "tennis", "ball", "running", "swimming", "cycling"],
    "art": ["vase","painting", "sculpture", "drawing", "museum", "gallery", "street art","dome"],
    "animal": ["animal", "dog", "cat", "bird", "fish", "wildlife","honeycomb",],
    "object": ["table", "balloon", "object", "chair", "lamp", "computer","basket"],
    "food": ["hotdog","food", "vegetable", "fruit", "meal", "dessert", "drink","plate","matchstick","cup","parachute","rotisserie","barometer"],
    "clothes": ["shirt", "cloak", "sock", "pants", "dress", "hat", "shoes", "stockings"],
    "transportation": ["bridge","car", "train", "airplane", "bike", "boat", "road", "highway"],
    "urban": ["street", "alley", "urban", "city life", "market", "park","window","uniform","solar_dish","fence","stupa","palace"],
    # Add more themes as needed
}

# Embeddings pour les thèmes
theme_embeddings = {theme: model_language.encode(keywords, convert_to_tensor=True) for theme, keywords in themes.items()}

for image_name in image_files:
    image_path = os.path.join(image_folder, image_name)
    # Générer des tags prédits pour l'image
    predicted_tags = predict_tags(image_path, model)
    predicted_theme = associated_theme(predicted_tags[0],model_language,theme_embeddings)
    # Trouver l'entrée correspondante dans les métadonnées et mettre à jour les tags
    for entry in metadata:
        if entry["nom"] == image_name:
            entry["tags"] = predicted_theme
            break

# Sauvegarder les métadonnées mises à jour
with open(metadata_file, 'w') as f:
    json.dump(metadata, f, indent=4)

# Séléction des images favorites

In [ ]:
!pip install ipywidgets

In [ ]:
import ipywidgets as widgets
from ipywidgets import GridspecLayout
from os import listdir

In [ ]:
images = []

for file in listdir("images/unsplash-images-collection")[:50]:
    images.append("images/unsplash-images-collection/" + file)

checkboxes = [widgets.Checkbox(value=False, description='Favorite') for _ in range(len(images))]

# Create the GridspecLayout widget
layout = GridspecLayout(n_columns=4, n_rows=len(images)//2, width='1000px')
for i, (img, checkbox) in enumerate(zip(images, checkboxes)):
  file = open(img, "rb")
  image = file.read()
  image_widget = widgets.Image(
    value=image,
    format='png',
    width=100,
    height=100,
  )
  layout[i//2, 0+((i%2)*2)] = image_widget
  layout[i//2, 1+((i%2)*2)] = checkbox

# Button to get selected images
button = widgets.Button(description="Select")

# Output widget to display selected images
output = widgets.Output()

def load_metadata():
    metadata_file = 'image_metadata.json'
    with open(metadata_file, 'r') as f:
        return json.load(f)

def favori(paths):
    metadata = load_metadata()
    for entry in metadata:
        if entry["nom"] in paths:
            entry["favori"] = "yes"
    with open(metadata_file, 'w') as f:
        json.dump(metadata, f, indent=5)

def not_favori(paths):
    metadata = load_metadata()
    for entry in metadata:
        if entry["nom"] in paths:
            entry["favori"] = "no"
    with open(metadata_file, 'w') as f:
        json.dump(metadata, f, indent=5)

# Function to get selected images
def get_selected_images(btn):
    selected_paths = [images[i][34:] for i, checkbox in enumerate(checkboxes) if checkbox.value]
    not_selected_paths = [images[i][34:] for i, checkbox in enumerate(checkboxes) if not checkbox.value]
    with output:
        output.clear_output()
        favori(selected_paths)
        not_favori(not_selected_paths)
        print("Préférences enregistrées.")

# Link button click event to function
button.on_click(get_selected_images)

# Display the layout and button
display(layout, button, output)

# Création du dataset final

In [ ]:
!pip install pandas

In [ ]:
import json

import numpy as np
import pandas as pd

from sklearn.linear_model import Perceptron
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier

In [ ]:
with open("image_metadata.json", "r") as file:
    data = json.load(file)
    

dataframe = pd.json_normalize(data)
dataframe['aire_image'] = dataframe['taille'].apply(lambda x: x[0] * x[1])

mlb = MultiLabelBinarizer()
tags_encoded = mlb.fit_transform(dataframe['tags'])
tags_df = pd.DataFrame(tags_encoded, columns=mlb.classes_)

format_df = pd.get_dummies(dataframe['format'], prefix='format')
orientation_df = pd.get_dummies(dataframe['orientation'], prefix='orientation')

dataframe['dominant_color_r'] = dataframe['couleur_dominante'].apply(
    lambda x: x[0] if isinstance(x, list) else 0)
dataframe['dominant_color_g'] = dataframe['couleur_dominante'].apply(
    lambda x: x[1] if isinstance(x, list) else 0)
dataframe['dominant_color_b'] = dataframe['couleur_dominante'].apply(
     lambda x: x[2] if isinstance(x, list) else 0)

all_final_df = pd.concat([
    tags_df,
    format_df,
    orientation_df,
    dataframe[['aire_image', 'dominant_color_r', 'dominant_color_g', 'dominant_color_b']]
    ], axis=1)

dataframe_evalue = dataframe[dataframe['favori'] != 'n/a']
dataframe_non_evalue = dataframe[dataframe['favori'] == 'n/a']

labels_evalue = dataframe_evalue['favori'].map({'yes': 1, 'no': 0})

X_train = all_final_df.loc[dataframe_evalue.index]
y_train = labels_evalue

X_test = all_final_df.loc[dataframe_non_evalue.index]


svc = SVC(probability=True)
perceptron = Perceptron()
decision_tree = DecisionTreeClassifier()


svc.fit(X_train, y_train)
perceptron.fit(X_train, y_train)
decision_tree.fit(X_train, y_train)


predictions_tree = decision_tree.predict(X_test)
indices_liked = np.where(predictions_tree == 1)[0]
top_n = 10
recommended_indices = indices_liked[:top_n]
recommended_images = dataframe_non_evalue.iloc[recommended_indices]['nom'].values

# Affichage des images recommandées

In [ ]:
import ipywidgets as widgets
from ipywidgets import GridspecLayout

In [ ]:
layout = GridspecLayout(n_columns=4, n_rows=len(recommended_images)//4+1, width='1000px')

for i, img in enumerate(recommended_images):
    file = open("images/unsplash-images-collection/" + img, "rb")
    image = file.read()
    image_widget = widgets.Image(
        value=image,
        format='png',
        width=100,
        height=100,
    )
    layout[i//4, i%4] = image_widget

display(layout)